# Extracting DCT2 Coefficients using DFT

References:
- Ahmed, Nasir, T_ Natarajan, and Kamisetty R. Rao. "Discrete cosine transform." IEEE transactions on Computers 100.1 (2006): 90-93.
- Strang, Gilbert. "The discrete cosine transform." SIAM review 41.1 (1999): 135-147.

In [ ]:
import numpy as np
from scipy.fftpack import dct

$$
X(j) = \alpha[j] \cdot \operatorname{Re} \left\{ e^{-j\frac{j\pi}{2n}} \cdot Z_j \right\}, \quad j = 0, 1, \dots, n-1,
\\
\text{where } \alpha[0] = \sqrt{\frac{1}{N}}, \text{ and } \alpha[j] = \sqrt{\frac{2}{N}} \text{ for }  j \in \{ 1, 2, \dots , N-1\}
\\
\mathbf{Z} = \mathbf{F}_{2n} \cdot \hat{\mathbf{x}}, \text{ given that } \mathbf{F}_{2n} \in \mathbb{C}^{2n \times 2n} \text{ is the DFT matrix }.
$$

In [ ]:
x = np.array([1, 2, 3, 4, 5, 6, 7, 8])
n = len(x)

x_padded = np.concatenate((x, np.zeros(n)))  # Length 2n

X_fft = np.fft.fft(x_padded)

k = np.arange(n)
rotation = np.exp(-1j * np.pi * k / (2 * n))
dct2 = np.real(rotation * X_fft[k])

alpha = np.ones(n) * np.sqrt(2 / n)
alpha[0] = np.sqrt(1 / n)

dct2_orthonormal = (dct2) * alpha

x_reconstructed = dct(dct2_orthonormal, type=3, norm='ortho')

print("\nOriginal:", x)
print("\nPadded:", x_padded)
print("\nDCT-II coefficients (via FFT):", dct2)
print("\nReconstructed (via DCT-III):", x_reconstructed)


Original: [1 2 3 4 5 6 7 8]

Padded: [1. 2. 3. 4. 5. 6. 7. 8. 0. 0. 0. 0. 0. 0. 0. 0.]

DCT-II coefficients (via FFT): [ 3.60000000e+01 -1.28846460e+01  0.00000000e+00 -1.34690960e+00
 -4.44089210e-16 -4.01805807e-01  0.00000000e+00 -1.01404646e-01]

Reconstructed (via DCT-III): [1. 2. 3. 4. 5. 6. 7. 8.]


In [ ]:
import numpy as np
from scipy.fftpack import dct

x = np.array([1, 2, 3, 4, 5, 6, 7, 8])
n = len(x)

X_fft = np.fft.fft(x)

k = np.arange(n)
rotation = np.exp(-1j * np.pi * k / (2 * n))
dct2 = np.real(rotation * X_fft[k])

alpha = np.ones(n) * np.sqrt(2 / n)
alpha[0] = np.sqrt(1 / n)

dct2_orthonormal = (dct2) * alpha

x_reconstructed = dct(dct2_orthonormal, type=3, norm='ortho')

x_final = np.concatenate((x_reconstructed[::2], x_reconstructed[1::2][::-1]))

print("\nOriginal:", x)
print("\nDCT-II coefficients (via FFT):", dct2)
print("\nReconstructed (via DCT-III):", x_reconstructed)
print("\nPermuted:", x_final)


Original: [1 2 3 4 5 6 7 8]

DCT-II coefficients (via FFT): [ 36.          -2.03918232  -2.1647844   -2.40537955  -2.82842712
  -3.59990489  -5.22625186 -10.25166179]

Reconstructed (via DCT-III): [1. 8. 2. 7. 3. 6. 4. 5.]

Permuted: [1. 2. 3. 4. 5. 6. 7. 8.]


# Test Case: Generator Matrix Encoded Data

$$
\text{Generator matrix: } \quad \tilde{M}_{kj} = \left[ \left( \frac{w_0}{z_0} \right)^j \zeta^{kj} \right]_{k,j=0}^{n-1} \\
\text{DFT matrix: } \quad F_{kj} = \left[ \zeta^{kj} \right]_{k,j=0}^{n-1} \\
\text{where:} \quad \zeta = e^{- \frac{2\pi i}{n}}
$$

In [ ]:
def padded_generator_matrix(N, w0, z0):
    n = np.arange(N)
    k = n.reshape((N, 1))
    zeta = np.exp(-2j * np.pi / N)
    M_tilde = ((w0 / z0) ** n) * (zeta ** (k * n))
    return M_tilde

x = np.array([1, 2, 3, 4, 5, 6, 7, 8])
n = len(x)

w0 = 4
z0 = 3

M_tilde = padded_generator_matrix(n, w0, z0)
X_encoded = np.dot(M_tilde, x)

k = np.arange(n)
rotation = np.exp(-1j * np.pi * k / (2 * n))
dct2 = np.real(rotation * X_encoded[k])

alpha = np.ones(n) * np.sqrt(2 / n)
alpha[0] = np.sqrt(1 / n)

dct2_orthonormal = (dct2) * alpha

x_reconstructed = dct(dct2_orthonormal, type=3, norm='ortho')

x_final = np.concatenate((x_reconstructed[::2], x_reconstructed[1::2][::-1]))

D_hat_n = np.array([(z0 / w0) ** k for k in range(n)])
x_rescaled = D_hat_n * x_final

print("\nOriginal:", x)
print("\nDCT-II coefficients (via FFT):", dct2)
print("\nReconstructed (via DCT-III):", x_reconstructed)
print("\nPermuted:", x_final)
print("\nRescaled:", x_rescaled)


Original: [1 2 3 4 5 6 7 8]

DCT-II coefficients (via FFT): [158.83081847  21.49734745  -9.87335067 -18.85543959 -25.38374316
 -33.84998579 -48.96911092 -83.06639977]

Reconstructed (via DCT-III): [ 1.         59.93232739  2.66666667 39.33058985  5.33333333 25.28395062
  9.48148148 15.80246914]

Permuted: [ 1.          2.66666667  5.33333333  9.48148148 15.80246914 25.28395062
 39.33058985 59.93232739]

Rescaled: [1. 2. 3. 4. 5. 6. 7. 8.]
